<a href="https://colab.research.google.com/github/jmoro0408/sloped_pipe_volume/blob/main/many_plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO

* Add case where pipe is partially compeltely full

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
pd.options.mode.chained_assignment = None  # default='warn' #disable pandas companing about creating a copy of a df
#warnings.filterwarnings('ignore')

In [2]:
# initial values
initial_grade_percent = 0.1 #inital grade value
maximum_grade_value = 5 #maximum grade (%) to plot 
no_grades = 20 #grades will range equally between inital grade value and maximum grade value by this many points
pipe_length = 200
no_points_to_plot = 100
pipe_diameter = 2 
pipe_invert_start = 0 #pipe invert at pipe end
lower_liquid_level = 1 #liquid level at pipe end



In [3]:
pipe_chainage = np.linspace(0, pipe_length, no_points_to_plot).tolist()
pipe_chainage = [round(elem) for elem in pipe_chainage]

In [4]:
def calculate_tilt_angle(initial_grade_percent):
  tilt_angle = np.arctan(initial_grade_percent / 100)
  tilt_angle = np.degrees(tilt_angle)
  return tilt_angle

def calculate_pipe_upper_invert(pipe_invert_elev,tilt_angle,pipe_length):
  pipe_upper_invert = np.tan(np.radians(tilt_angle)) * pipe_length + pipe_invert_elev
  return pipe_upper_invert

def calculate_upper_liquid_level(lower_liquid_level, tilt_angle, pipe_length):
  upper_liquid_level = lower_liquid_level
  return upper_liquid_level

def calculate_sloped_pipe_volume(pipe_diameter, tilt_angle, pipe_length, lower_liquid_level):
  pipe_radius = pipe_diameter/2
  tilt_angle=np.radians(tilt_angle)
  _upper_liquid_level = lower_liquid_level - (pipe_length *np.tan(tilt_angle))

  # "normal" case, where both ends have some liquid
  if _upper_liquid_level > 0 :

    _h0 = _upper_liquid_level
    _lf = pipe_length
  
  # upper end has no liquid - liquid does not reach end of pipe
  else:
    #lower_liquid_level = 0
    _lf = pipe_length + _upper_liquid_level/np.tan(tilt_angle)
    _h0 = 0
    assert _lf < pipe_length, "Liquid fill length cannot be greater than pipe length"

  _h1 = _h0 + (_lf * np.tan(tilt_angle))
  # Case where pipe is (at least) partially completely full. 
  if _h1 > pipe_diameter:
    _lt = (_h1 - pipe_diameter) / np.tan(tilt_angle)
  else:
    _lt = 0

  _pipe_length = _lf - _lt
  assert _h0 >= 0, "_h0 must be positive"

  _k = 1 - (_h0/pipe_radius)
  _c = _k - ((_pipe_length * np.tan(tilt_angle))/pipe_radius)
  
  integral0 = pipe_radius**3 / np.tan(tilt_angle)
  integral1 = _k* np.arccos(_k)
  integral2 = (1/3) * (np.sqrt(1 - (_k**2))) * (_k**2 +2)
  integral3 = _c * np.arccos(_c)
  integral4 = (1/3) * (np.sqrt(1 - (_c**2))) * (_c**2 +2)
  volume = integral0 * (integral1 - integral2 - integral3 + integral4)
  
  if (pipe_radius == lower_liquid_level) and (tilt_angle == 0): #Pipe is exactly half full and no slope
    volume = (pipe_length * np.pi * pipe_radius **2)/2
  return volume


In [5]:
tilt_angle = calculate_tilt_angle(initial_grade_percent)
upper_pipe_invert = calculate_pipe_upper_invert(pipe_invert_start,tilt_angle,pipe_length)
upper_liquid_level = calculate_upper_liquid_level(lower_liquid_level, tilt_angle, pipe_length)

In [6]:
#create x points along the chainage
pipe_invert_points = np.linspace(pipe_invert_start, upper_pipe_invert, no_points_to_plot)

# multiple plots

Need to create plot lines for each grade percentage in slider list

In [7]:
initial_grade_percent_list = np.linspace(0, maximum_grade_value, no_grades) #this will be the slider eventually
initial_grade_percent_list = np.round(initial_grade_percent_list, 2)

In [8]:
#Building a dictionary that holdes the grade and every respective pipe invert y coordinate 
pipe_invert_dict = {}
for grade in initial_grade_percent_list:
  _tilt_angle = calculate_tilt_angle(grade)
  _upper_pipe_invert = calculate_pipe_upper_invert(pipe_invert_start, _tilt_angle,pipe_length)
  _pipe_invert_points = np.linspace(pipe_invert_start, _upper_pipe_invert, no_points_to_plot)
  _pipe_invert_points = np.round(_pipe_invert_points, 2)
  _pipe_invert_points = _pipe_invert_points.tolist()
  pipe_invert_dict.update({grade:_pipe_invert_points})


In [9]:
#Building a dictionary that holdes the grade and every respective pipe crown y coordinate 
# The pipe crown is just the invert + the pipe diameter, so this is easy to build from the invert dict
pipe_crown_dict = {}
for key, values in pipe_invert_dict.items():
  _crown_value = [value + pipe_diameter for value in values]
  _crown_value = [round(value, 2) for value in _crown_value]
  pipe_crown_dict.update({key:_crown_value})


In [10]:
# Building a dict to hold the respective upper liquid levels at every grade. This is just equal to the lower level as the water level does not move
liquid_level_dict = {}
for grade in initial_grade_percent_list:
  liquid_level_dict.update({grade: [lower_liquid_level] * no_points_to_plot})

# for grade in initial_grade_percent_list:
#   _tilt_angle = calculate_tilt_angle(grade)
#   _upper_liquid_level = calculate_upper_liquid_level(lower_liquid_level, _tilt_angle, pipe_length)
#   _upper_liquid_level_points = np.linspace(lower_liquid_level,_upper_liquid_level,  no_points_to_plot)
#   _upper_liquid_level_points = _upper_liquid_level_points.tolist()
#   liquid_level_dict.update({grade:_upper_liquid_level_points})



In [11]:
"""this one is tricky. Trying to make sure any point where the liquid level is below the pipe invert is not plotted. 

I think there is a neater way to do this but I tried going directly to the go.Figure() object, and to the individual dicts but
got lost navigating the nested stucture. This way works for now. 
"""

pipe_invert_df = pd.DataFrame.from_dict(pipe_invert_dict) #create dataframes from both dicts
liquid_level_df = pd.DataFrame.from_dict(liquid_level_dict)

for column in pipe_invert_df:
  for index in pipe_invert_df.index:
    if liquid_level_df[column].iloc[index] < pipe_invert_df[column].iloc[index]: # if liquid level < pipe invert, replace with NaN
      liquid_level_df[column].iloc[index] = np.nan

liquid_level_dict = liquid_level_df.to_dict(orient = "list")

del pipe_invert_df, liquid_level_df #dont need these dataframes anymore

In [12]:
fig1 = go.Figure()
trace_list1 = []
trace_list2 = []
trace_list3 = []
for grade in pipe_invert_dict.keys():
  _tilt_angle = round(calculate_tilt_angle(grade),2)
  trace_list1.append(go.Scatter(visible = False, 
                                x=pipe_chainage, 
                                y=pipe_invert_dict[grade], 
                                name = "Pipe Invert",
                                showlegend=False, 
                                line=dict(color='gray', width=2)))
  
  trace_list2.append(go.Scatter(visible = False, 
                                x=pipe_chainage, 
                                y=pipe_crown_dict[grade], 
                                name = "Pipe Crown",
                                showlegend=False, 
                                line=dict(color='gray', width=2)))
  
  trace_list3.append(go.Scatter(visible = False, 
                                x=pipe_chainage, 
                                y=liquid_level_dict[grade], 
                                name = "Liquid Level", 
                                line=dict(color='royalblue', width=2)))

In [13]:
#Building the slider
# in order to capture both the crown and invert moving together with a singler slider step we need to append both traces to a single list

num_steps = len(initial_grade_percent_list) #each step = a different grade
fig2 = go.Figure(data = trace_list1 + trace_list2+trace_list3)

#have to be really careful here. You want the slider to be aware of the ordering of traces, see https://community.plotly.com/t/multiple-traces-with-a-single-slider-in-plotly/16356

fig2.data[0].visible = True #this is the pipe invert data
fig2.data[num_steps].visible = True #pipe crown data
fig2.data[num_steps*2].visible = True #liquid level data


# Set initial slider/title index
start_index = 0

steps = []
for i in range(num_steps):
  current_grade_value = round(initial_grade_percent_list[i], 2)
  step = dict(
      method="update", 
      args = [
              {"visible":[False] * len(fig2.data)}, 
              {"title":f"Liquid Level Along Pipe at {current_grade_value} % Grade"}, 
              ], 
              label=str(round(list(pipe_invert_dict.keys())[i],2)))
  
  step["args"][start_index]["visible"][i] = True  # Toggle i'th trace to "visible" - pipe invert
  step["args"][start_index]["visible"][i+num_steps] = True #pipe crown
  step["args"][start_index]["visible"][i+num_steps*2] = True #liquid level
  steps.append(step)

sliders = [dict( 
    active=initial_grade_percent,
    currentvalue={"prefix": "Grade: ", "suffix": " %"},
    pad={"t": 50},
    steps=steps
)]

fig2.update_layout(
    template = "simple_white", 
    title=f"Liquid Level Along Pipe at {start_index} % Grade",
    xaxis_title="Pipe Chainage (m)",
    yaxis_title="Elevation (mGeodetic)",
    sliders=sliders,
  #Legend info 
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1),
)



fig2.update_yaxes(range=[min(min(pipe_invert_dict.values()))-1,max(max(pipe_crown_dict.values()))]) #have to lock the y axis, otherwise the axis just updates and the line doesnt "move"
fig2.update_xaxes(showspikes=True, spikecolor="green", spikesnap="cursor", spikemode="across")
fig2.update_yaxes(showspikes=True,spikecolor="green")

fig2.show()



# Plotting air/water volume percentage

In [14]:
def calculate_total_pipe_volume(pipe_diameter, pipe_length):
  pipe_cross_sectional_area = np.pi / 4 * (pipe_diameter**2)
  total_volume = pipe_cross_sectional_area * pipe_length
  total_volume = round(total_volume, 2)
  return total_volume


In [15]:
#Building a doct that holds the pipe volume for each grade %
liquid_volume_grade_dict = {}
for key, value in liquid_level_dict.items():
  _pipe_radius = pipe_diameter/2
  _tilt_angle = calculate_tilt_angle(key)
  _pipe_volume = calculate_sloped_pipe_volume(pipe_diameter, _tilt_angle, pipe_length, lower_liquid_level)
  liquid_volume_grade_dict.update({key:_pipe_volume})


liquid_volume_grade_dict = {key: round(liquid_volume_grade_dict[key], 2) for key in liquid_volume_grade_dict} # rouding grades for plotting

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning:

divide by zero encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: RuntimeWarning:

invalid value encountered in double_scalars



In [16]:
pipe_total_volume = calculate_total_pipe_volume(pipe_diameter, pipe_length)

In [17]:
air_volume_dict = {} # Volume of air is just toal volume - volume of liquid
for key, value in liquid_volume_grade_dict.items():
  air_volume_dict.update({key: pipe_total_volume-value})
  air_volume_dict = {key: round(air_volume_dict[key], 2) for key in air_volume_dict}

In [18]:
air_liquid_percent_dict = {}
for key, value in liquid_volume_grade_dict.items():
  _liquid_percent = (liquid_volume_grade_dict[key] / pipe_total_volume) *100
  _liquid_percent = round(_liquid_percent,2)
  _air_percent = 100 - _liquid_percent
  _air_percent = round(_air_percent, 2)
  air_liquid_percent_dict.update({key: [_liquid_percent, _air_percent]})


In [22]:
# two lists of air percents and liquid percents makes plotting a whole easier, so just use list comprehension to explode the dict
air_percents_list = [list(air_liquid_percent_dict.values())[i][1] for i in range(len(air_liquid_percent_dict.values()))]
liquid_percents_list = [list(air_liquid_percent_dict.values())[i][0] for i in range(len(air_liquid_percent_dict.values()))]


fig3 = make_subplots(specs=[[{"secondary_y": True}]])
fig3.add_trace(go.Scatter(x=list(air_liquid_percent_dict.keys()), y= air_percents_list, name = "%  Air",showlegend=True, line=dict(color='orange', width=2)))
fig3.add_trace(go.Scatter(x=list(air_liquid_percent_dict.keys()), y= liquid_percents_list, name = "% Liquid",showlegend=True, line=dict(color='royalblue', width=2)))

fig3.add_trace(go.Scatter(x=list(air_volume_dict.keys()), y= list(air_volume_dict.values()), name = "Volume air",showlegend=True, line=dict(color='orange', width=2, dash = "dot")), secondary_y = True)
fig3.add_trace(go.Scatter(x=list(liquid_volume_grade_dict.keys()), y= list(liquid_volume_grade_dict.values()), name = "Volume liquid",showlegend=True, line=dict(color='royalblue', width=2, dash = "dot")), secondary_y = True)

fig3.update_xaxes(showspikes=True, spikecolor="green",spikesnap = "cursor", spikemode="across", spikethickness = 2)
fig3.update_yaxes(showspikes=True,spikecolor="green",spikesnap = "cursor", spikemode="across", spikethickness = 2)


fig3.update_layout(
    template = "simple_white", 
    title="Air/Liquid Volume Split at Various Grades",
    xaxis_title="Grade (%)",

    yaxis=dict(
    title="% Pipe Volume",
    side="right", 
    range = [0,100]
  ),
  yaxis2=dict(
    title=r"$\text {Pipe Volume} \: (m^3)$",
    anchor="free",
    overlaying="y",
    side="left", 
  ),
  #Legend info 
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1),
)


